# 1. 환경세팅
requirements: conda

밑의 모든 코드는 EmoSDS/ 디렉토리에서 실행할 것

## 1-1. Llama 3.2 3B 모델 다운로드

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

!mkdir llama/3_2/3B
!cd llama/3_2/3B
!git clone https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
!cd ../../../

## 1-2. kmeans 모델 다운로드

In [ ]:
# >> 여기서 pt파일 다운받아서 utils/speech2unit 에 저장할 것.
# https://www.notion.so/jhwanlee/Future-Internet-16dd543c7357803d96e6f9b7914048a3?pvs=4#175d543c735780dfac64fdd9e5ebe44a

## 1-3. conda 환경 생성

In [ ]:
!conda create --name emosds python=3.8
!conda activate emosds
!conda install pip=24.0
!conda install cudatoolkit=11.7 -c conda-forge
!pip install -r requirements.txt
!pip install -U datasets
# pip install transformers==4.34
!pip install omegaconf==2.3.0
!pip install accelerate==0.26.0
!pip install torch==2.4.1
!pip install torchaudio==2.4.1
!pip install transformers==4.45
!pip install --upgrade huggingface-hub
!pip install bert_score

# 2. 데이터 Preprocess

## 2-1. ASR task 데이터 생성

In [ ]:
# >> EmoSDS 디렉토리에서 실행
!python3 utils/build_data.py asr
!python3 utils/merge_jsonl_files.py data/asr_task_styletalk_train.jsonl data/asr_task_styletalk_eval.jsonl data/asr_task_styletalk_train_eval.jsonl
!python3 utils/merge_jsonl_files.py data/asr_task_styletalk_train_eval.jsonl data/asr_task_librispeech.jsonl data/asr_task_librispeech_styletalk.jsonl

## 2-1. Unified(ASR + Style Prediction + Response Prediction) task 데이터 생성

In [ ]:
# >> EmoSDS 디렉토리에서 실행
!python3 utils/build_data.py unified

# 3. ASR task 훈련 진행

In [ ]:
# >> EmoSDS 디렉토리에서 실행
!bash scripts/asr_sft.sh

In [ ]:
# >> 모델 zip파일로 저장(파일만 저장함)
#    EmoSDS 디렉토리에서 실행
# ! 실행하기 전에 zip 파일명 바꿀 것

!find /shared/NAS_SSD/jhl/futureinternet/output/asr -type f -maxdepth 1 -exec zip asr_training_6epochs_20250113.zip {} +

# 4. Unified(ASR + Style Prediction + Response Prediction) task 훈련 진행

In [ ]:
# >> EmoSDS 디렉토리에서 실행
!bash scripts/unified_sft.sh

In [ ]:
# >> 모델 zip파일로 저장(파일만 저장함)
#    EmoSDS 디렉토리에서 실행
# ! 실행하기 전에 zip 파일명 바꿀 것

!find /shared/NAS_SSD/jhl/futureinternet/output/unified -type f -maxdepth 1 -exec zip unified_training_6epochs_20250113.zip {} +

# 5. CLI inference 수행

In [ ]:
# >> EmoSDS 디렉토리에서 실행
!python3 src/infer/cli_infer_emosds.py --model-path "/shared/NAS_SSD/jhl/futureinternet/output/stage3/checkpoint-3600" --output-dir "output"

# 입력 prompt 형식: "[History] <<Input>>: [wav파일path(assets/testprompts)]"